## Train linear  model om predictions

### Training the model

In [1]:
import os
import bcolz
from utils import get_data,get_batches
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

from keras import backend as K

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 Ti (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [20]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [ ]:
#path = "data/dogscats/sample/"
path = "data/dogscats/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

In [22]:
# batch_size = 4
batch_size = 100

In [23]:
batches = vgg.get_batches(path+'train', shuffle=False, batch_size=1)
val_batches = vgg.get_batches(path+'valid', shuffle=False, batch_size=1)
# test_batches = vgg.get_batches(path+'test', shuffle=False, batch_size=1)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [24]:
def save_array(fname, arr):
    c = bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

In [25]:
val_data = get_data(path+'valid')

Found 8 images belonging to 2 classes.


In [26]:
trn_data = get_data(path+'train')

Found 16 images belonging to 2 classes.


In [27]:
trn_data.shape

(16, 3, 224, 224)

In [28]:
save_array(model_path+'train_data.bc', trn_data)
save_array(model_path+'valid_data.bc', val_data)

In [29]:
trn_data = load_array(model_path+'train_data.bc')
val_data = load_array(model_path+'valid_data.bc')

In [30]:
def onehot(x):
    return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [31]:
val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [32]:
trn_labels.shape

(16, 2)

In [33]:
trn_classes[:4]

array([0, 0, 0, 0], dtype=int32)

In [34]:
trn_labels[:4]

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [35]:
trn_features = model.predict(trn_data, batch_size=batch_size)
val_features = model.predict(val_data, batch_size=batch_size)

In [36]:
trn_features.shape

(16, 1000)

In [37]:
save_array(model_path+'train_lastlayer_features.bc', trn_features)
save_array(model_path+'valid_lastlayer_features.bc', val_features)

In [38]:
trn_features = load_array(model_path+'train_lastlayer_features.bc')
val_features = load_array(model_path+'valid_lastlayer_features.bc')

In [39]:
lm = Sequential([ Dense(2, activation='softmax', input_shape = (1000,)) ])
lm.compile(optimizer=RMSprop(lr=0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [40]:
lm.fit(trn_features, trn_labels, nb_epoch=3, batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 16 samples, validate on 8 samples
Epoch 1/3
16/16 [==============================] - 0s - loss: 0.6941 - acc: 0.3125 - val_loss: 0.5583 - val_acc: 0.5000
Epoch 2/3
16/16 [==============================] - 0s - loss: 0.5424 - acc: 0.5625 - val_loss: 0.4680 - val_acc: 1.0000
Epoch 3/3
16/16 [==============================] - 0s - loss: 0.4093 - acc: 1.0000 - val_loss: 0.4005 - val_acc: 1.0000


In [41]:
lm.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
dense_7 (Dense)                  (None, 2)             2002        dense_input_1[0][0]              
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
____________________________________________________________________________________________________


### Viewing model prediction examples

it is not important of this part,skip to next topic.

# Modifying the model

## Retrain last layer's linear model

In [42]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

In [43]:
model.pop()
for layer in model.layers:
    layer.trainable=False

In [44]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

you can see the last layer has remove. and then, we add a new layer at last layer.

In [45]:
model.add(Dense(2, activation='softmax'))

In [46]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_2 (Lambda)                (None, 3, 224, 224)   0           lambda_input_2[0][0]             
____________________________________________________________________________________________________
zeropadding2d_14 (ZeroPadding2D) (None, 3, 226, 226)   0           lambda_2[0][0]                   
____________________________________________________________________________________________________
convolution2d_14 (Convolution2D) (None, 64, 224, 224)  1792        zeropadding2d_14[0][0]           
____________________________________________________________________________________________________
zeropadding2d_15 (ZeroPadding2D) (None, 64, 226, 226)  0           convolution2d_14[0][0]           
___________________________________________________________________________________________

## Image Data Augmention

In [47]:
gen = image.ImageDataGenerator()
batches = gen.flow(trn_data, trn_labels, batch_size=batch_size, shuffle=True)
val_batches = gen.flow(val_data, val_labels, batch_size=batch_size, shuffle=True)

In [48]:
opt = RMSprop(lr=0.1)
model.compile(optimizer=opt, loss='categorical_crossentropy', matrics=['accuracy'])

In [49]:
def fit_model(model, batches, val_batches, nb_epoch=1):
    model.fit_generator(batches, samples_per_epoch=batches.n, nb_epoch=nb_epoch, validation_data=val_batches,nb_val_samples=val_batches.n)

In [50]:
fit_model(model, batches, val_batches, nb_epoch=2)

/home/syh/.conda/envs/fastai/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1960: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Epoch 1/2
16/16 [==============================] - 5s - loss: 1.9090 - val_loss: 2.0148
Epoch 2/2
16/16 [==============================] - 5s - loss: 1.0074 - val_loss: 2.0148


In [51]:
model.save_weights(model_path+'finetune1.h5')
model.load_weights(model_path+'finetune1.h5')

In [52]:
model.evaluate(val_data,val_labels)

8/8 [==============================] - 1s


2.0147619247436523

In [53]:
preds = model.predict_classes(val_data, batch_size=batch_size)
probs = model.predict_proba(val_data, batch_size=batch_size)[:,0]

probs[:8]

/home/syh/.conda/envs/fastai/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:1960: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


8/8 [==============================] - 1s


array([  1.0000e+00,   1.0000e+00,   1.0000e+00,   1.0000e+00,   9.8825e-28,   0.0000e+00,
         8.9658e-26,   1.0000e+00], dtype=float32)

## Training mutiple layers in Keras

In [55]:
layers = model.layers
first_dense_idx = [index for index, layer in enumerate(layers) if type(layer) is Dense][0]

for layer in layers[first_dense_idx:]: layer.trainable=True

In [56]:
K.set_value(opt.lr, 0.01)
fit_model(model, batches, val_batches, 3)

Epoch 1/3
16/16 [==============================] - 5s - loss: 2.0148 - val_loss: 2.0148
Epoch 2/3
16/16 [==============================] - 5s - loss: 1.0074 - val_loss: 2.0148
Epoch 3/3
16/16 [==============================] - 5s - loss: 2.0148 - val_loss: 2.0148


In [57]:
model.save_weights(model_path+'finetune2.h5')

In [58]:
for layer in layers[12:]:
    layer.treanable = True
K.set_value(opt.lr, 0.001)

In [59]:
fit_model(model, batches, val_batches, 4)

Epoch 1/4
16/16 [==============================] - 5s - loss: 1.0074 - val_loss: 2.0148
Epoch 2/4
16/16 [==============================] - 5s - loss: 2.0148 - val_loss: 2.0148
Epoch 3/4
16/16 [==============================] - 6s - loss: 1.0074 - val_loss: 2.0148
Epoch 4/4
16/16 [==============================] - 6s - loss: 2.0148 - val_loss: 2.0148


In [60]:
model.save_weights(model_path+'finetune3.h5')

In [63]:
model.load_weights(model_path+'finetune2.h5')
model.evaluate_generator(get_batches(path+'valid', gen, False, batch_size*2), val_batches.n)

Found 8 images belonging to 2 classes.


2.0147619247436523